# Fortran refresher

## Introduction

Fortran was developed in the 1950's as a purpose built language for STEM applications. It is a good language to use for high performance computing because it is **easy** to understand, highly performant, and comes with a first-rate array implementation. Furthermore, the language has access to both thread-level and accelerator-assisted paralleism via extensions such as OpenMP and OpenACC, as well as access to process-level parallelism with a message passing library.

It is true that older versions of the Fortran standard (i.e Fortran 77 and earlier) have archaic features and idiosyncrasies that may discourage folks with training in modern coding techniques. However, like other standard programming languages such as C++, the standard is under active development, and modern Fortran programs can even be expressed in object-oriented coding styles. Some of the benefits include:

* Longevity, Fortran will be around as a standard for a long time.
* First-rate multi-dimensional array implementation. Arrays of up to seven dimensions are supported, with easy **numpy-like** array access.
* High performance. Compilers can optimize Fortran code so that it runs very quickly.
* Compiler-optimized memory management. Compilers are free to organise and manage memory allocations in optimal ways.

Some potential drawbacks include:

* Objects are passed to subroutines and functions by reference as the default. This potentially has impacts for memory safety.
* Poor string handling and concatenation.
* Memory alignment complexities when interfacing Fortran with C
* No central website for the Fortran standard
* Finding expertise in Fortran is difficult

## Teaching method

This teaching module does not aim to provide an exhaustive introduction to Fortran. Instead we aim to cover concepts that are helpful when using Fortran code with the [Hipfort](https://rocm.docs.amd.com/projects/hipfort/en/latest/) interface to AMD's [HIP](https://rocm.docs.amd.com/projects/HIP/en/latest/) GPU library and runtime. We do this using a simple tensor addition example. If **A**, **B**, and **C** are tensors of rank 1, then at each index $i$ the following equation will be valid:

$$
\textbf{C}(i) = \textbf{A}(i) + \textbf{B}(i)
$$

We solve this problem a number of ways by example, using Fortran programs that progressively demonstrate features available to the Fortran language. Programmers from other languages will then readily be able to map concepts to their language of expertise. We choose to follow the modern 2008 standard and disregard archaic language features such as fixed source form. 

## Fortran basics

In the program [tensoradd_simple.f90](tensoradd_simple.f90) is a simple example of a Fortran program that peforms tensor addition. Let's open this by clicking on it and go through it line by line. 

### Program definition

Every Fortran program has one (and only one) **program** construct followed by the name of the program. 

```Fortran
program tensoradd
```

at the end of the program there is a corresponding **end** statement to signify the end of the program. 

```Fortran
end program tensoradd
```

It is good practice to also **include the name** of that which is being ended, in this case we are ending **program tensoradd**.

### Comments

Comments in Fortran start with a `!`, anything beyond the `!` is normally disregarded by the compiler if it doesn't form part of some directive like an OpenMP construct. The [FORD](https://forddocs.readthedocs.io/en/latest/) documentation generator uses a double `!!` to signify comments that should be in the documentation. 

```Fortran
   !! Program to compute a 1D tensor addition
   !! Written by Dr. Toby Potter and Dr. Joseph Schoonover
```

Other comments are to provide assistance to other programmers, most likely your **future self**. In literate programming, attributed to [Donald Knuth](https://en.wikipedia.org/wiki/Literate_programming), code can be thought of as an explanation to the compiler of what the comment is saying.

### Variable names

In Fortran all variables must be declared **before any statements**. This means declaring all the variables we are going to use at the beginning, as opposed to when they are needed. Variable names must begin with a letter and may contain letters `[a-zA-Z]`, numbers `[0-9]`, or an underscore `_`. Fortran is also **case insensitive** by default, meaning that uppercase and lowercase code is interpreted the same way unless you set a compiler flag.

### Implicit (weak) typing

By default Fortran also has implicit typing. This means that if a variable starts with `i, j, k, l, m, n` then a variable is interpreted as a (32-bit) integer, otherwise it is interpreted a (32-bit) floating point number. It is good practice to make sure that no implicit typing occurs with the `implicit none` statement.

```Fortran
    ! Add this to make sure that all variables must be declared
    ! and the compiler performs no type inferencing based on the 
    ! on the first letter of variable names
    implicit none
```

### Basic Fortran data types

There are a number of basic data types in Fortran. Each data type has a default number of bytes that it uses, as shown in the table below but has options for varying the number of bytes employed. 

| Data type | default number of bytes |
|:--:|:--:|
| integer | 4 |
| real | 4 |
| complex | 2x4 |
| logical | 4 |
| character | 1 |

Compiler flags can be used to vary the number of bytes used as the default for integers and reals, however this is **not a good idea** when data is being transferred between systems, such as to a GPU. It is better to use predetermined types with a fixed number of bytes. The [ISO_FORTRTAN_ENV](https://fortranwiki.org/fortran/show/iso_fortran_env) module defines a number of standard data types with fixed numbers of bytes. It is advisable to use these data types when having a standard number of bits to represent your data is a priority.

### Declaring variables

Declaring a variable is the done by specifying the variable type, with any options followed by `::` then the variable name and an optionally an equals sign `=` for variable initialisation. Here we create an integer `N` with a value of 16. The option `parameter` tells the compiler that the variable is fixed as a parameter and is effectively read-only.

```Fortran
integer, parameter :: N=16
```

Similarly, we define a `eps_mult` as a real with a value of 2.0.

```Fortran
real :: eps_mult = 2.0
```

Logical values are either `.true.` or `.false.` the value `success` is to contain wether or not the validation was successful. Here we set it to true unlesss proven otherwise.

```Fortran
logical :: success = .true.
```

#### Declaring static arrays

Arrays/tensors may be declared statically on the stack or allocated dynamically on the heap. Arrays of up to 7 dimensions specified by putting comman-separated tuple of dimensions after the variable name. Here we declare 3 tensors **A_h**, **B_h**, and **C_h**, each with 1 dimension and a length of N.

```Fortran
    ! Declare the tensors to use
    ! Memory for these will be allocated on the stack
    real :: A_h(N), B_h(N), C_h(N)
```

### Array indexing convention

Array indices start at 1 by default. These can be modified at declaration/allocation by using a colon `:` to specify the lower and upper indices of each dimension of the array. For example, in order for the tensors to start with index 0 we could have done this instead.

```Fortran
    ! Declare the tensors to use
    ! Memory for these will be allocated on the stack
    real :: A_h(0:N), B_h(0:N), C_h(0:N)
```

### Call subroutines and functions

Fortran has **functions** and **subroutines**. Functions take any number of input variables and return 1 output variable. The output of functions can be used on the right hand side of an assignment statement. For example, the call to the built-in **spacing** function returns the value of floating point spacing from the input floating point value to the next value. Here we use it in the validation step to help calculate the upper and lower bounds on a validation result.

```Fortran
upper = scratch + eps_mult*spacing(abs(scratch))
lower = scratch - eps_mult*spacing(abs(scratch))
```

**Subroutines** are like functions but they do not return a value. Here we use the built-in subroutine **random_number** to fill the tensors **A_h** and **B_h** with random numbers.

```Fortran
    ! Fill arrays with random numbers using the
    ! Fortran intrinsic function "random_number"
    call random_number(A_h)
    call random_number(B_h)
```

> Note: For research applications of random number generation it is recommended to investigate the random number generator further and make sure that you have a high quality one.

Variables passed to functions and subroutines are generally passed **by reference** as the default. This means a reference/pointer to the variable is passed in and it is possible to modify the input arguments from the function or subroutine. For example the call to **random_number** above modifies the tensors **A_h** in **B_h** in place. To avoid ambiguity it is **best practice** to design functions so they do not modify the input arguments and use subroutines when the contents of input arguments are to be modified.

### Loops and array access

Loops are made using the `do` construct. Here we loop over i from 1 to N inclusive and set elements in **C_h** using elements in **A_h** and **B_h**. Indexing into arrays is done by appending a comma-separated tuple of coordinates to the array name.

```Fortran
do i=1,N
    ! Kernel math
    C_h(i) = A_h(i) + B_h(i)
end do
```

Note that unlike C, which allows the loop variable to be declared in the loop construct, in Fortran we must declare all variables at the beginning of the program, function, or subroutine. For non-deterministic loops there is the corresponding `do while` loop. The above could then be expressed as:

```Fortran
i=1
do while (i<=N)
    ! Kernel math
    C_h(i) = A_h(i) + B_h(i)
    i = i + 1
end do
```

In Fortran we can also perform bulk operations on portions of arrays using the colon operator `:` in a way that is similar to Numpy. We can achieve the same operation on the tensors using this method: 

```Fortran
    ! Could also do it this way, (best practice)
    C_h(:) = A_h(:) + B_h(: )
```

Or we can even do it this way, though for readability it is not preferred because we lose a sense of what rank the tensors are.

```Fortran
    ! Or even this way (not best practice)
    C_h = A_h + B_h
```

Loops can of course be nested. If this was a two dimensional problem it would look something like this:

```Fortran
do j=1,M
    do j=1,N
        ! Kernel math
        C_h(i,j) = A_h(i,j) + B_h(i,j)
    end do
end do
```

Or using indexing operations the above can be written as:

```Fortran
C_h(:,:) = A_h(:,:) + B_h(:,: )
```

#### Array ordering

Arrays in Fortran are arranged in **column-major** format. This means that array elements are contiguous along the **first** dimension of the array. In CPU code this means that you will be able to make best use of cache lines if the **innermost loop** traverses the first dimension of your arrays. With GPU code it is good for performance to make sure that **neighbouring threads** along the first dimension of the Grid are associated with neighbouring elements along the first dimension of the array. You are free to iterate along any of the other dimensions within a kernel and neighbouring threads can share cache lines read in from memory. 

### Line continuation

Unlike C and C++, Fortran doesn't have an end of statement character such as `;` in C, so by default the compiler interprets a new line to mean a new statement. You can modify this and spill a statement to the next line by using the **&** line continuation character.